### Q1. You are working on a machine learning project where you have a dataset containing numerical and categorical features. You have identified that some of the features are highly correlated and there are missing values in some of the columns. You want to build a pipeline that automates the feature engineering process and handles the missing values.

Design a pipeline that includes the following steps:


- Use an automated feature selection method to identify the Important features in the dataset.

- Create a numerical pipeline that includes the following steps:

- Impute the missing values in the numerical columns using the mean of the column values.

- Scale the numerical columns using standardisation.

- Create a categorical pipeline that includes the following steps:

- Impute the missing values in the categorical columns using the most frequent value of the column.

- One-hot encode the categorical columns.

- Combine the numerical and categorical pipelines using a Column Transformer.

- Use a Random Forest Classifier to build the final model.

- Evaluate the accuracy of the model on the test dataset.


Note: Your solution should include code snippets for each step of the pipeline, and a brief explanation of

each step. You should also provide an interpretation of the results and suggest possible improvements for

the pipeline.




In [1]:
import seaborn as sns 
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
X=df.drop(columns='tip')
y=df['tip']

In [3]:
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

In [4]:
# # Step 1: Automated feature selection using SelectFromModel

# feature_selector = SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=42))
# feature_selector.fit(X, y)
# selected_features = X.columns[feature_selector.get_support()]


In [5]:
df.dtypes

total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
dtype: object

In [6]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer

In [7]:
#  Numerical pipeline
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [8]:
# categorical_pipeline  

categorical_features=X.select_dtypes(include=['category']).columns
categorical_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder())
])

In [9]:
# Combine numerical and categorical pipelines using ColumnTransformer
from sklearn.compose import ColumnTransformer
preprocessor=ColumnTransformer(
    transformers=[
        ('num',numerical_pipeline,numerical_features),
        ('cat',categorical_pipeline,categorical_features)
])

In [10]:
# Final pipeline with RandomForestRegressor

final_pipeline=Pipeline([
('preprocessor',preprocessor),
('regressor',RandomForestRegressor(n_estimators=100, random_state=42))
])

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3, random_state=42)


In [12]:
final_pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['total_bill', 'size'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['sex', 'smoker', 'day', 'time'], dtype='object'))])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [13]:
y_pred=final_pipeline.predict(X_test)

In [14]:
from sklearn.metrics import r2_score

In [15]:
print(r2_score(y_test,y_pred))

0.26694893270866993


### Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.

In [16]:
from sklearn.datasets import load_iris

In [17]:
iris=load_iris()

In [18]:
X,y=iris.data,iris.target

In [ ]:
from sklearn.model_selection import train_test_split

In [19]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=42)

In [20]:
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression

In [22]:
rf_classifier=RandomForestClassifier(n_estimators=100,random_state=42)
lr_classifier=LogisticRegression(max_iter=100,random_state=42)

In [23]:
from sklearn.pipeline import Pipeline

In [24]:
pipeline=Pipeline([
    ('scaler',StandardScaler()),
    ('voting_classifier',VotingClassifier(estimators=[
        ('random_forest',rf_classifier),
        ('logistic_regression',lr_classifier)
    ],voting='hard'))
])

In [25]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('voting_classifier',
                 VotingClassifier(estimators=[('random_forest',
                                               RandomForestClassifier(random_state=42)),
                                              ('logistic_regression',
                                               LogisticRegression(random_state=42))]))])

In [26]:
y_pred=pipeline.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score

In [28]:
accuracy_score(y_test,y_pred)

1.0